In [2]:
import matplotlib.pyplot as plt
import rospy
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, VBox, HBox, Label
from assignment_2_2023.msg import CustomStatus
from assignment_2_2023.srv import Input, Goal
from turtlesim.msg import Pose
from matplotlib.animation import FuncAnimation
import threading
import time

In [3]:
class Visualiser:
    def _setup_world(self):
        # box border
        self.ax1.plot([9.5,-9.5,-9.5,9.5,9.5],[9.5,9.5,-9.5,-9.5,9.5],'y')
        # obstacles
        self.ax1.plot([4, -5, -5], [-0.5, -0.5, -5], 'brown')
        self.ax1.plot([-7,5], [4.5,4.5], 'brown')
        
    def _set_goal_status(self):
        counter = 0
        while(True):
            goal = self.get_goal()
            output_display.value = goal.status
            if not self.first_exec and goal.status != self._old_goal_status:
                if goal.status == 'Reached':
                    self.reached += 1
                    self._update_pie(True)
                elif goal.status == 'Canceled':
                    self.canceled += 1
                    self._update_pie(False)
            if self.first_exec:
                self.first_exec = False
            self._old_goal_status = goal.status
            time.sleep(1)
    
    def __init__(self):
        #self.fig, self.ax = plt.subplots()
        self.fig = plt.figure()
        self.ax1 = self.fig.add_subplot(121)
        self.ln, = plt.plot([], [], 'b-')
        self.ax2 = self.fig.add_subplot(122)
        self.ax2.pie([0,0], labels=['Reached', 'Canceled'])
        self.x_data, self.y_data = [], []
        self.reached, self.canceled = 0,0
        self.runner_thread = None
        self._old_goal_status = None
        self.first_exec = True
        self.send_input = rospy.ServiceProxy('/assignment2_2023/input_string', Input)
        self.get_goal = rospy.ServiceProxy('/assignment_2_2023/last_goal', Goal)


    def plot_init(self):
        self.ax1.set_xlim(-10, 10)
        self.ax1.set_ylim(-10, 10)
        self.ax1.set_title("Assignment display")
        self.ax1.set_xlabel("x")
        self.ax1.set_ylabel("y")
        self.ax1.invert_xaxis()
        self.ax1.invert_yaxis()
        self.ax1.set_aspect('equal', adjustable='box')
        self._setup_world()
        return self.ln,

    def odom_callback(self, msg):
        self.y_data.append(msg.y)
        self.x_data.append(msg.x)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln,
    
    def on_send_goal_pressed(self, a):
        self.send_input("("+str(next_position_x.value)+","+str(next_position_y.value)+")")
        self.ax1.plot(next_position_x.value, next_position_y.value, 'go')
    
    def on_cancel_pressed(self, b):
        goal = self.get_goal()
        self.send_input("cancel")
        self.ax1.plot(goal.goal_x, goal.goal_y, 'ro')


    def activate_goal_status_reception(self):
        self.runner_thread = threading.Thread(target=self._set_goal_status)
        self.runner_thread.start()
        
    def _update_pie(self, success):
        self.ax2.clear()
        self.ax1.set_title(success)
        if success :
            self.reached += 1
        else:
            self.canceled += 1
        self.ax2.pie([self.reached, self.canceled], labels=['Reached', 'Canceled'])
        #self.fig.canvas.draw_idle()
            

In [4]:
next_position_x = widgets.FloatText(
    value=1,
    description='X:',
    disabled=False,
    layout=Layout(width = '150px', align = "left")
)

next_position_y = widgets.FloatText(
    value=1,
    description='Y:',
    disabled=False,
    layout = Layout(width = '150px')
)

send_goal = Button(
    description='Send Goal',
    style=ButtonStyle(button_color='lightgreen')
)

cancel = Button(
    description='Cancel',
    layout=Layout(width='auto', align="center"),
    style=ButtonStyle(button_color='red')
)

output_display = widgets.Text(description='Status: ', disabled=True)

a = VBox(
    [
        HBox([
                Label("Please enter the next goal -> "),
                next_position_x, 
                next_position_y, 
                send_goal, 
                cancel
            ]),
        output_display,
        Label("Position graph:"),
    ]
)

display(a)

In [5]:
rospy.wait_for_service('/assignment2_2023/input_string');
rospy.wait_for_service('/assignment_2_2023/last_goal');

%matplotlib widget
rospy.init_node('Assignment2_RT2')
vis = Visualiser()
send_goal.on_click(vis.on_send_goal_pressed)
cancel.on_click(vis.on_cancel_pressed)
sub = rospy.Subscriber('assignment_2_2023/customStatus', CustomStatus, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
vis.activate_goal_status_reception()
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …